In [19]:
from pomegranate import *
import numpy as np

# 몬티홀 게임쇼


Monty Hall 문제는 게임쇼 Let's Make Deal에서 게스트가 세 개의 문 중 어떤 문 뒤에 상이 있는지 선택해야 하는 게임에서 발생했습니다.
반전은 손님이 선택한 후 원래 Monty Hall이었던 호스트가 손님이 선택하지 않은 문 중 하나를 열고 손님이 선택한 문을 전환하기를 원하는지 묻습니다.
초기 검사를 통해 문이 두 개뿐인 경우 올바른 문을 고를 확률이 50-50이므로 어떤 이점도 없다고 믿게 될 수 있습니다.
그러나 시뮬레이션과 분석을 통해 게스트가 처음에 갔던 문과 상관없이 손님이 문을 바꾸면 실제로 상품을 받을 확률이 66%라는 것이 입증되었습니다.

우리는 3개의 노드가 있는 베이지안 네트워크를 사용하여 이 결과를 재현할 수 있습니다. 하나는 게스트, 하나는 상금, 하나는 Monty가 열기로 선택한 문입니다.
손님이 처음 선택하는 문과 상품 뒤에 있는 문은 세 개의 문에 걸쳐 완전히 무작위 프로세스이지만 Monty가 여는 문은 손님이 선택하는 문(손님이 선택한 문이 될 수 없음)에 따라 다릅니다.
상품이 뒤에 있는 문입니다(상품이 뒤에 있는 문일 수 없음).

석류에 베이지안 네트워크를 생성하려면 먼저 그래프의 각 노드에 있는 분포를 생성합니다.
이산(범주형) 베이지안 네트워크의 경우 루트 노드에는 DiscreteDistribution 개체를 사용하고 내부 및 리프 노드에는 ConditionalProbabilityTable 개체를 사용합니다.
ConditionalProbabilityTable의 열은 부모(두 번째 인수)가 지정된 순서에 해당하며 마지막 열은 ConditionalProbabilityTable 자체가 취하는 값입니다.
아래의 경우 첫 번째 열은 'guest' 값, 그 다음 'prize' 값, 'monty' 값에 해당합니다.
'B', 'C', 'A'는 손님이 문 'B'를 선택했고 상품이 실제로 문 'C' 뒤에 있는 경우 Monty가 문 'A'를 드러낼 확률 또는 P(Monty= 'A'|손님='B', 상품='C').

In [2]:
# The guests initial door selection is completely random
guest = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})

# The door the prize is behind is also completely random
prize = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})

# Monty is dependent on both the guest and the prize.
monty = ConditionalProbabilityTable(
    [[ 'A', 'A', 'A', 0.0 ],
     [ 'A', 'A', 'B', 0.5 ],
     [ 'A', 'A', 'C', 0.5 ],
     [ 'A', 'B', 'A', 0.0 ],
     [ 'A', 'B', 'B', 0.0 ],
     [ 'A', 'B', 'C', 1.0 ],
     [ 'A', 'C', 'A', 0.0 ],
     [ 'A', 'C', 'B', 1.0 ],
     [ 'A', 'C', 'C', 0.0 ],
     [ 'B', 'A', 'A', 0.0 ],
     [ 'B', 'A', 'B', 0.0 ],
     [ 'B', 'A', 'C', 1.0 ],
     [ 'B', 'B', 'A', 0.5 ],
     [ 'B', 'B', 'B', 0.0 ],
     [ 'B', 'B', 'C', 0.5 ],
     [ 'B', 'C', 'A', 1.0 ],
     [ 'B', 'C', 'B', 0.0 ],
     [ 'B', 'C', 'C', 0.0 ],
     [ 'C', 'A', 'A', 0.0 ],
     [ 'C', 'A', 'B', 1.0 ],
     [ 'C', 'A', 'C', 0.0 ],
     [ 'C', 'B', 'A', 1.0 ],
     [ 'C', 'B', 'B', 0.0 ],
     [ 'C', 'B', 'C', 0.0 ],
     [ 'C', 'C', 'A', 0.5 ],
     [ 'C', 'C', 'B', 0.5 ],
     [ 'C', 'C', 'C', 0.0 ]], [guest, prize])

In [9]:
s1 = Node(guest, name="guest")
s2 = Node(prize, name="prize")
s3 = Node(monty, name="monty")

In [10]:
print(s1)
print(s2)
print(s3)

{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "A" : 0.3333333333333333,
                "B" : 0.3333333333333333,
                "C" : 0.3333333333333333
            }
        ],
        "frozen" : false
    },
    "name" : "guest",
    "weight" : 1.0
}
{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "A" : 0.3333333333333333,
                "B" : 0.3333333333333333,
                "C" : 0.3333333333333333
            }
        ],
        "frozen" : false
    },
    "name" : "prize",
    "weight" : 1.0
}
{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "name" : "ConditionalProbabilityTable",
        "table" : [
            [
            

In [11]:
model = BayesianNetwork("Monty Hall Problem")
model.add_states(s1, s2, s3)
model.add_edge(s1, s3)
model.add_edge(s2, s3)
model.bake()

In [22]:
model.probability([['A', 'B', 'C']])

0.11111111111111109

In [23]:

model.probability([['A', 'B', 'B']])

0.0

In [16]:
print(model.probability([['A', 'A', 'A'],
                         ['A', 'A', 'B'],
                         ['C', 'C', 'B']]))

[0.         0.05555556 0.05555556]


In [21]:
print(model.predict([['A', 'B', None],
                     ['A', None, 'C'],
                     [None, 'B', 'A']]))

[array(['A', 'B', 'C'], dtype=object), array(['A', 'B', 'C'], dtype=object), array(['C', 'B', 'A'], dtype=object)]


C:\Users\pwjdg\AppData\Local\Temp\ipykernel_16728\1168950580.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print(model.predict([['A', 'B', None],


In [20]:
print(model.predict([['A', 'B', None],
                     ['A', 'C', None],
                     ['C', 'B', None]]))

[array(['A', 'B', 'C'], dtype=object), array(['A', 'C', 'B'], dtype=object), array(['C', 'B', 'A'], dtype=object)]


C:\Users\pwjdg\AppData\Local\Temp\ipykernel_16728\2013110763.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print(model.predict([['A', 'B', None],
